Code to predict type of pmeumonia from exray of chest of patient.

In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

from os import listdir
from os.path import isfile, join
import numpy as np
import tensorflow as tf
from PIL import Image as img
from tensorflow.python import keras
from keras import Sequential, datasets, layers, models
import pickle as pk

Various imports needed.

In [4]:
pixels = 150
dimensions = (pixels, pixels)
bins = 2
mid_layer = 750
training_iters = 10
learning_rate = 2e-3
num_epochs = 1
reduction_factor = 0.9

Variables defined

In [5]:
class LearningRateReducerCb(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        old_lr = self.model.optimizer.lr.read_value()
        new_lr = old_lr * reduction_factor
        print("\nEpoch: {}. Reducing Learning Rate from {} to {}".format(epoch, old_lr, new_lr))
        self.model.optimizer.lr.assign(new_lr)

Reduces learning rate by epochs.

In [6]:
train_images = []
train_labels = np.array([])
folder_path = "/mnt/c/NN/Pneumonia/chest_xray/train/"

for f in listdir(folder_path + "NORMAL"):
    im = img.open(folder_path + "NORMAL/" + f)
    im = im.resize(dimensions)
    im = im.convert('L')
    im_array = np.asarray(im)
    train_images.append(im_array)
    im.close()
    train_labels = np.append(train_labels, 0)

num_normal = np.size(train_labels)

for f in listdir(folder_path + "PNEUMONIA"):
    im = img.open(folder_path + "PNEUMONIA/" + f)
    im = im.resize(dimensions)
    im = im.convert('L')
    im_array = np.asarray(im)
    train_images.append(im_array)
    im.close()

    train_labels = np.append(train_labels, 1)    
    
    if np.size(train_labels) >= 2 * num_normal:
        break
    

test_images = []
test_labels = np.array([])
folder_path = "/mnt/c/NN/Pneumonia/chest_xray/test/"

for f in listdir(folder_path + "NORMAL"):
    im = img.open(folder_path + "NORMAL/" + f)
    im = im.resize(dimensions)
    im = im.convert('L')
    im_array = np.asarray(im)
    test_images.append(im_array)
    im.close()    
    test_labels = np.append(test_labels, 0)

for f in listdir(folder_path + "PNEUMONIA"):
    im = img.open(folder_path + "PNEUMONIA/" + f)
    im = im.resize(dimensions)
    im = im.convert('L')
    im_array = np.asarray(im)
    test_images.append(im_array)
    im.close()

    test_labels = np.append(test_labels, 1)    
    # if "bacteria" in f:
        # test_labels = np.append(test_labels, 1)
    # elif "virus" in f:
        # test_labels = np.append(test_labels, 2)

train_images = np.array(train_images)
test_images = np.array(test_images)

train_images = train_images / 255.0
test_images = test_images / 255.0

In [ ]:
unique, counts = np.unique(train_labels, return_counts=True)
print(dict(zip(unique, counts)))

Images Imported
Average time taken = 150 seconds.

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(pixels, (3, 3), activation='relu', input_shape=(pixels, pixels, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(2 * pixels, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(2 * pixels, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(pixels, activation='relu'))
model.add(layers.Dense(bins))

opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=opt,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

Creates CNN model

In [ ]:
history = model.fit(train_images, train_labels, callbacks=[LearningRateReducerCb()], epochs=num_epochs,
                    validation_data=(test_images, test_labels))

Trains the model.
Number of epochs = 3
Training time = 33:27

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print("Accuracy = " + str(np.round(test_acc * 100, 3)) + " %")

Prints accuracy for the model.
Validation Accuracy reached = 65

In [ ]:
filename = 'model.sav'
pk.dump(model, open(filename, 'wb'))

In [ ]:
img.fromarray(train_images[0] * 255).show()
to_predict = np.reshape(train_images[0:1341], (1341, pixels, pixels, 1))
# to_predict = np.expand_dims(to_predict, 0)
download_model = pk.load(open('model.sav', 'rb'))
temp = download_model.predict(to_predict)
print(temp.count(0))
print(temp.count(1))
# print(download_model.predict(to_predict))

In [21]:
image = img.open("/mnt/c/NN/Pneumonia/chest_xray/train/NORMAL/IM-0115-0001.jpeg")
image = image.resize(dimensions)
image = image.convert('L')
image_array = np.asarray(image)
image_array = np.reshape(image_array, (pixels, pixels, 1))
image_array = np.expand_dims(image_array, 0)
image_array = image_array / 255.0

print(download_model.predict(image_array))

[[ 1.2432315 -2.343593 ]]
